# Import the necessary packages.

In [3]:
import torch 
from torch import nn, optim 
from torch.autograd import Variable 
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy

import pytorch_lightning as pl 
from pytorch_lightning.callbacks import ModelCheckpoint

In [4]:
print('torch version: ', torch.__version__)
print('pytorch lightning version: ', pl.__version__)

torch version:  1.12.1+cu116
pytorch lightning version:  1.7.7


In [5]:
xor_input = [
    Variable(torch.Tensor([0, 0])),
    Variable(torch.Tensor([0, 1])),
    Variable(torch.Tensor([1, 0])),
    Variable(torch.Tensor([1, 1]))
]

In [6]:
xor_target = [
    Variable(torch.Tensor([0])),
    Variable(torch.Tensor([1])),
    Variable(torch.Tensor([1])),
    Variable(torch.Tensor([0]))
]

In [7]:
xor_data = list(zip(xor_input, xor_target))
train_loader = DataLoader(xor_data, batch_size=1000)

In [39]:
class XORModel(pl.LightningModule):
    def __init__(self):
        super(XORModel, self).__init__()

        self.input_layer = nn.Linear(2, 4)
        self.output_layer = nn.Linear(4, 1)

        self.sigmoid = nn.Sigmoid()
        self.loss = nn.MSELoss()
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.sigmoid(x)
        x = self.output_layer(x)
        return x
    
    def configure_optimizers(self):
        params = self.parameters()
        optimizer = optim.Adam(params=params, lr=0.01)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        xor_input, xor_target = batch 
        outputs = self(xor_input)
        loss = self.loss(outputs, xor_target)
        return loss 

In [46]:
checkpoint_callback = ModelCheckpoint()
model = XORModel()
trainer = pl.Trainer(default_root_dir='xor_model_ckpts' , max_epochs=1000, callbacks=[checkpoint_callback])
trainer.fit(model, train_dataloaders=train_loader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type    | Params
-----------------------------------------
0 | input_layer  | Linear  | 12    
1 | output_layer | Linear  | 5     
2 | sigmoid      | Sigmoid | 0     
3 | loss         | MSELoss | 0     
-----------------------------------------
17        Trainable params
0         Non-trainable params
17        Total params
0.000     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1000` reached.


In [12]:
print(checkpoint_callback.best_model_path)

xor_model/lightning_logs/version_0/checkpoints/epoch=499-step=500.ckpt


In [48]:
print(checkpoint_callback.best_model_path)

train_model = model.load_from_checkpoint(checkpoint_path=checkpoint_callback.best_model_path)
test_loader = DataLoader(xor_input, batch_size=1)

for batch in test_loader:
    preds = train_model(batch)
    for i in range(batch.shape[0]):
        print([int(batch[i][0]),int(batch[i][1])], int(preds[i].round()))

xor_model/lightning_logs/version_1/checkpoints/epoch=999-step=1000.ckpt
[0, 0] 0
[0, 1] 1
[1, 0] 1
[1, 1] 0


In [47]:
print(checkpoint_callback.best_model_path)

train_model = model.load_from_checkpoint(checkpoint_path=checkpoint_callback.best_model_path)
total_accuracy = []
for xor_input, xor_target in train_loader:
    for i in range(100):
        output_tensor = train_model(xor_input)
        test_accuracy = accuracy(output_tensor, xor_target.round().int())
        total_accuracy.append(test_accuracy)
total_accuracy = torch.mean(torch.stack(total_accuracy))
print("Total accuracy for 100 iterations: ", total_accuracy.item())

xor_model/lightning_logs/version_1/checkpoints/epoch=999-step=1000.ckpt
Total accuracy for 100 iterations:  1.0
